9/24-9/26

This is more code for the Kaggle Ames, Iowa Housing Prices Competition. Last time, I tried to prune some features and make my own. With a normal linear regression model, this led to a horrible test error of 0.6. Obviously, pruning some features isn't the right move. Now I'm only going to make features, and only delete the features that I make my features from, and then use LASSO. Maybe some form of boosting if I want to improve my results further.

In [1]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, learning_curve, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import locale
from scipy.stats import randint
locale.setlocale( locale.LC_ALL, '' )

'English_United States.1252'

In [2]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [3]:
prices = pd.DataFrame(train_set["SalePrice"].copy())
nec_data = train_set.drop("SalePrice", axis=1) # labels
ids = test_set["Id"].copy()

In [4]:
# dropping features with too many null values
nec_data = nec_data.drop("Alley", axis=1)
nec_data = nec_data.drop("PoolQC", axis=1) 
nec_data = nec_data.drop("Fence", axis=1)  
nec_data = nec_data.drop("MiscFeature", axis=1)
nec_data = nec_data.drop("Id", axis=1) # too individual

# making new features
nec_data["has_porch"] = (nec_data["OpenPorchSF"] + nec_data["EnclosedPorch"] + nec_data["3SsnPorch"] + nec_data["ScreenPorch"] > 0)
nec_data["has_deck"] = (nec_data["WoodDeckSF"] > 0)
nec_data["has_pool"] = (nec_data["PoolArea"] > 0)
nec_data["TotalSF"] = nec_data["1stFlrSF"] + nec_data["2ndFlrSF"] + nec_data["TotalBsmtSF"]

# used in self-made feature
nec_data = nec_data.drop("1stFlrSF", axis=1) 
nec_data = nec_data.drop("2ndFlrSF", axis=1) 
nec_data = nec_data.drop("TotalBsmtSF", axis=1) 
nec_data = nec_data.drop("GrLivArea", axis=1) 

nec_data = nec_data.drop("WoodDeckSF", axis=1) 

nec_data = nec_data.drop("PoolArea", axis=1) 

nec_data = nec_data.drop("OpenPorchSF", axis=1) 
nec_data = nec_data.drop("EnclosedPorch", axis=1) 
nec_data = nec_data.drop("3SsnPorch", axis=1) 
nec_data = nec_data.drop("ScreenPorch", axis=1) 

In [5]:
# grouping objects that need ordinal encoding or one-hot encoding
ordinal_features = ["LotShape", "LandSlope", "HeatingQC", "Functional", "BsmtFinType1"]
col_name_list = nec_data.columns.values.tolist()
for name in col_name_list:
    if name[-4:] == "Qual" or name[-4:] == "Cond":
        ordinal_features.append(name)
one_hot_features = np.setdiff1d(list(nec_data.select_dtypes(include=['object']).columns), ordinal_features)

log_features = ["MSSubClass", "LotFrontage", "LotArea", "YearBuilt", "TotalSF"]

In [6]:
def safe_log(x):
    return np.log(x + 1e-10)
#thanks to ChatGPT 3.5 for this function
log_transformer = FunctionTransformer(func=safe_log, inverse_func=np.exp)

num_tail_pipeline = make_pipeline(SimpleImputer(strategy="median"), log_transformer, StandardScaler())
category_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))
ranking_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OrdinalEncoder())
num_norm_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

preprocess = ColumnTransformer([
    ("tail", num_tail_pipeline, log_features),
    ("category", category_pipeline, make_column_selector(dtype_include=object))],
    remainder=num_norm_pipeline
)

In [7]:
lin_reg = Pipeline([("pre", preprocess), ("reg", Lasso(alpha=0.1, max_iter=10000, random_state=446))])
lin_reg.fit(nec_data, prices)

housing_predictions = lin_reg.predict(nec_data)
for k in range(10):
    print(f"Real value: {locale.currency(prices.iloc[k].values[0], grouping=True)}, Prediction: {locale.currency(housing_predictions[k].round(2), grouping=True)}")

Real value: $208,500.00, Prediction: $210,788.02
Real value: $181,500.00, Prediction: $210,814.72
Real value: $223,500.00, Prediction: $207,199.17
Real value: $140,000.00, Prediction: $158,058.16
Real value: $250,000.00, Prediction: $301,858.04
Real value: $143,000.00, Prediction: $144,881.20
Real value: $307,000.00, Prediction: $284,873.92
Real value: $200,000.00, Prediction: $214,674.50
Real value: $129,900.00, Prediction: $141,008.30
Real value: $118,000.00, Prediction: $120,728.11


In [8]:
lin_rmse = mean_squared_error(prices, housing_predictions, squared=False)
print(lin_rmse)

rmse = -cross_val_score(lin_reg, nec_data, prices, scoring="neg_root_mean_squared_error", cv=5)
print(np.average(rmse))

22520.21483434068


c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5797127029.486755, tolerance: 759214014.2531197
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3782728864.4177856, tolerance: 699031751.5199751
  model = cd_fast.sparse_enet_coordinate_descent(


33279.53161267178


c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13733563214.049805, tolerance: 724930391.3116124
  model = cd_fast.sparse_enet_coordinate_descent(


That's a pretty big difference between the train and validation accuracy, $11,000. Maybe random search and/or using elastic kit regression will help decrease that difference.

In [9]:
lin_reg2 = Pipeline([("pre", preprocess), ("reg", LassoCV(max_iter=10000, random_state=446))])
lin_reg2.fit(nec_data, prices)

housing_predictions2 = lin_reg2.predict(nec_data)
lin_rmse2 = mean_squared_error(prices, housing_predictions2, squared=False)
print(lin_rmse2)
rmse2 = -cross_val_score(lin_reg2, nec_data, prices, scoring="neg_root_mean_squared_error", cv=5)
print(np.average(rmse2))

c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


25776.142402712998


c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed w

29818.480258101463


$29,818 is better than $33,279, and the difference is also down to about $4,000. This warrants another competition submission.

In [10]:
# test_set["has_porch"] = (test_set["OpenPorchSF"] + test_set["EnclosedPorch"] + test_set["3SsnPorch"] + test_set["ScreenPorch"] > 0)
# test_set["has_deck"] = (test_set["WoodDeckSF"] > 0)
# test_set["has_pool"] = (test_set["PoolArea"] > 0)
# test_set["TotalSF"] = test_set["1stFlrSF"] + test_set["2ndFlrSF"] + test_set["TotalBsmtSF"]

# test_pred = lin_reg2.predict(test_set)

# with open('sacreddeer_house_new_submission_4.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(["Id", "SalePrice"])
#     for k in range(len(ids)):
#         writer.writerow([ids[k], test_pred[k]])

Test error of 0.33, still very far above my lowest error of 0.2. I might have to go back to using PCA/truncated SVD, but for the moment I want to futher explore different models.

In [11]:
elast_reg = Pipeline([("pre", preprocess), ("reg", ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=10000, random_state=446))])
elast_reg.fit(nec_data, prices)

housing_predictions3 = elast_reg.predict(nec_data)
lin_rmse3 = mean_squared_error(prices, housing_predictions3, squared=False)
print(lin_rmse3)
rmse3 = -cross_val_score(elast_reg, nec_data, np.array(prices).ravel(), scoring="neg_root_mean_squared_error", cv=5)
print(np.average(rmse3))

c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


25776.142402712998
29818.480258101463


Looks like the best model was just Lasso again, since the MSE and CV score were the same as with LassoCV. It's time to gradient boosting, then decision trees, then random forests.

In [12]:
print(lin_reg2["reg"].alpha_)

145.11318737159658


In [13]:
grad_lasso_reg = Pipeline([("pre", preprocess), ("reg", LassoCV(max_iter=10000, random_state=446))])
grad_lasso_reg.fit(nec_data, prices)

housing_predictions = grad_lasso_reg.predict(nec_data)
lin_rmse4 = mean_squared_error(prices, housing_predictions, squared=False)
print(lin_rmse4)
rmse4 = -cross_val_score(grad_lasso_reg, nec_data, np.array(prices).ravel(), scoring="neg_root_mean_squared_error", cv=5)
print(np.average(rmse4))

c:\Users\user_02\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


25776.142402712998
29818.480258101463


Same values as last time.

In [14]:
dec_tree_reg = Pipeline([("pre", preprocess), ("reg", DecisionTreeClassifier(random_state=446))])
dec_tree_reg.fit(nec_data, prices)

housing_predictions = dec_tree_reg.predict(nec_data)
lin_rmse5 = mean_squared_error(prices, housing_predictions, squared=False)
print(lin_rmse5)
rmse5 = -cross_val_score(dec_tree_reg, nec_data, np.array(prices).ravel(), scoring="neg_root_mean_squared_error", cv=5)
print(np.average(rmse5))

0.0


c:\Users\user_02\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


53286.176551494034


In [15]:
randdepth = np.array([2, 3, 4, 5, 6, 7, 8, 9])
param_distribs = {"reg__max_depth": randdepth,
                  "reg__min_samples_leaf": randint(low=1, high=20)}
rmd_search = RandomizedSearchCV(
    dec_tree_reg, param_distributions=param_distribs, 
    n_iter=20, cv=5, scoring="neg_root_mean_squared_error", random_state=446
)
rmd_search.fit(nec_data, prices)
final_rnd_model = rmd_search.best_estimator_
print(final_rnd_model.get_params)

final_rnd_model.fit(nec_data, prices)
rmse6 = -cross_val_score(final_rnd_model, nec_data, np.array(prices).ravel(), scoring="neg_root_mean_squared_error", cv=10)
print(np.average(rmse6))

c:\Users\user_02\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


<bound method Pipeline.get_params of Pipeline(steps=[('pre',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('tail',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(func=<function safe_log at 0x0000014CF8A82DD0>,
                                                                                       inverse_func=<ufun...
                      

c:\Users\user_02\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


51160.49311358758


In [16]:
rand_forest_reg = Pipeline([("pre", preprocess), ("reg", RandomForestRegressor(random_state=446, oob_score=True))])
rand_forest_reg.fit(nec_data, prices)

housing_predictions = rand_forest_reg.predict(nec_data)
lin_rmse7 = mean_squared_error(prices, housing_predictions, squared=False)
print(lin_rmse7)
rmse7 = -cross_val_score(rand_forest_reg, nec_data, np.array(prices).ravel(), scoring="neg_root_mean_squared_error", cv=5)
print(np.average(rmse7))

c:\Users\user_02\anaconda3\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


10614.881018560249
30114.770131979778


In [17]:
param_distribs2 = {"reg__max_depth": randdepth,
                  "reg__min_samples_leaf": randint(low=1, high=10),
                  "reg__n_estimators": randint(low=100, high=400)}
rmd_search2 = RandomizedSearchCV(
    rand_forest_reg, param_distributions=param_distribs2, 
    n_iter=10, cv=5, scoring="neg_root_mean_squared_error", random_state=446
)
rmd_search2.fit(nec_data, np.array(prices).ravel())
final_frt_model = rmd_search2.best_estimator_
print(final_frt_model.get_params)

<bound method Pipeline.get_params of Pipeline(steps=[('pre',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('tail',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(func=<function safe_log at 0x0000014CF8A82DD0>,
                                                                                       inverse_func=<ufun...
                      

In [18]:
final_frt_model.fit(nec_data, prices)
print(final_frt_model["reg"].oob_score_)

c:\Users\user_02\anaconda3\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8560395317448578


In [19]:
rmse8 = -cross_val_score(final_frt_model, nec_data, np.array(prices).ravel(), scoring="neg_root_mean_squared_error", cv=5)
print(np.average(rmse8))

30009.075382417785


In [20]:
housing_predictions = final_frt_model.predict(nec_data)
lin_rmse8 = mean_squared_error(prices, housing_predictions, squared=False)
print(lin_rmse8)

13194.207518756275
